### Notebook for solar panel prediction

In [ ]:
# Imports
%matplotlib inline
import matplotlib.pyplot as plt
from keras.layers import *
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.applications.resnet50 import preprocess_input
from keras.optimizers import Adam

###### First we define our deep-learning model:

In [ ]:
inputs = Input(shape=(256, 256, 3))
x = BatchNormalization()(inputs)
x = Conv2D(filters=16, kernel_size=3, activation='relu')(x)
x = BatchNormalization()(x)
x = Conv2D(filters=16, kernel_size=3, activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPool2D()(x)
x = Conv2D(filters=32, kernel_size=3, activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPool2D()(x)
x = Conv2D(filters=32, kernel_size=3, activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPool2D()(x)
x = Conv2D(filters=64, kernel_size=3, activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPool2D()(x)
x = Conv2D(filters=128, kernel_size=3, activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPool2D()(x)
x = Flatten()(x)

x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
x = Dense(2, activation='softmax')(x)
model = Model(inputs=inputs, outputs=x)
model.compile(Adam(lr=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

###### We create image generators that generate train and validation data

In [ ]:
batch_size = 32
train_gen = ImageDataGenerator(preprocessing_function=preprocess_input, zoom_range=0.1, width_shift_range=10,
                               rotation_range=90, vertical_flip=True, horizontal_flip=True, height_shift_range=10,
                               fill_mode='reflect').flow_from_directory('../satdata/train', batch_size=batch_size)

val_gen = ImageDataGenerator(preprocessing_function=preprocess_input).flow_from_directory('../satdata/val', shuffle=False, batch_size=batch_size)

###### We train our deep learning model

In [ ]:
model.fit_generator(train_gen, train_gen.n // batch_size, epochs=2, verbose=1, workers=16, use_multiprocessing=True,
                    validation_data=train_gen, validation_steps=val_gen.n // batch_size)

##### Testing our model

In [ ]:
batch_size = 1
model = load_model("../new_model.hdf5")
test_gen = ImageDataGenerator(preprocessing_function=preprocess_input).flow_from_directory('../satdata/test_wijchen', shuffle=False, batch_size=batch_size)

In [ ]:
preds = []
i = 0
for image, name in zip(test_gen, test_gen.filenames):
    pred = model.predict(image[0])[0, 1]
    preds.append((name, pred))
    if pred > 0.97:
        i += 1
        img = load_img(f"../satdata/test_wijchen/{name}")

        plt.imshow(img)
        plt.show()
        print(f"{name}: {pred:.2f}")
        
test_gen.reset()
print(i)

In [ ]:
import pickle
pickle.dump(preds, open('../predictions.pkl', 'wb'))

In [ ]:
from IPython.display import FileLink
FileLink('../predictions.pkl')

In [ ]:
model.save('../new_model.hdf5')